In [74]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
from keras.layers import Add
from keras.layers import Input
from keras.models import Model
from keras.layers import Flatten

In [87]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger,EarlyStopping,ModelCheckpoint

In [80]:
x_train = np.load("x_train_out.npy")
y_train = np.load("y_train_out.npy")
x_test = np.load("x_test_out.npy")
y_test = np.load("y_test_out.npy")

In [81]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [82]:
#z-score

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [83]:
#one hot encoding of target labels
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [63]:
#Initial convolutional block it will be common for all WRN-D-K networks
def initial_conv(Input):
    x = Conv2D(16,kernel_size=(3,3),strides = (1,1),padding = 'same')(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [64]:
#Expand Convolutional block having extra convolutional layer in Identity side

"""
Conv_no : Used to increase the depth of the network according to corresponding conv block
Stride : Used for Downsampling of Image
Dropout : Adds dropout if it is greater than 0.0

"""
def expand_conv(Input, k=1, conv_no = 1,stride=1,dropout =0.0):
    Initial = Input
    x = Conv2D(32*k*conv_no,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if(dropout > 0):
        x = Dropout(dropout)(x)
    x = Conv2D(32*k*conv_no,kernel_size=(3,3),strides = (1,1),padding ='same')(x)
    skip = Conv2D(32*k*conv_no,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(Input)
    output = Add()([x,skip])
    return output

In [1]:
#First Convolutional Block 
#Input size = 32*32*N

"""
K : To Control the depth of residual network
Stride : Used for Downsampling of Image
Dropout : Adds dropout if it is greater than 0.0

"""
def conv_block1(Input, k = 1, stride = 1, dropout = 0.0):
    Initial = Input 
    x = BatchNormalization()(Input)
    x = Activation('relu')(x)
    x = Conv2D(32*k,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32*k,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(x)
    output = Add()([x,Initial])
    return output

In [66]:
#Second Convolutional Block 
#Input size = 16*16*N

"""
K : To Control the depth of residual network
Stride : Used for Downsampling of Image
Dropout : Adds dropout if it is greater than 0.0

"""
def conv_block2(Input, k = 1, stride = 1, dropout = 0.0):
    Initial = Input 
    x = BatchNormalization()(Input)
    x = Activation('relu')(x)
    x = Conv2D(64*k,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64*k,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(x)
    output = Add()([x,Initial])
    return output

In [67]:
#Third Convolutional Block 
#Input size = 8*8*N

"""
K : To Control the depth of residual network
Stride : Used for Downsampling of Image
Dropout : Adds dropout if it is greater than 0.0

"""
def conv_block3(Input, k = 1, stride = 1, dropout = 0.0):
    Initial = Input 
    x = BatchNormalization()(Input)
    x = Activation('relu')(x)
    x = Conv2D(128*k,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128*k,kernel_size=(3,3),strides = (stride,stride),padding = 'same')(x)
    output = Add()([x,Initial])
    return output

In [75]:
"""
Creates Wide Residual network with given parameters

height = height of the Image
width = width of the Image
K = Width of the network(will be multiplied)
depth = No of convolutional layers

"""
def create_wide_residual_network(height,width,k,depth):
    
    
    """
    n = depth of the network
    Depth of the network. n = (n - 4) / 6.
              Example : For a depth of 16, n = 16, N = (16 - 4) / 6 = 2
              Example2: For a depth of 28, n = 28, N = (28 - 4) / 6 = 4
              Example3: For a depth of 40, n = 40, N = (40 - 4) / 6 = 6
              
    """
    n = (depth-4)/6
    
    #Input tensor
    inputs = Input(shape = (height,width,3), name = "image_input")
    
    #Initial Conv block
    x = initial_conv(inputs)
    
    #First Expansion block
    x = expand_conv(x,k)
    
    #First Convolutional Block
    #Depth depends on n
    for i in range(n-1):
        x = conv_block1(x,k)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #Second Expansion Block
    x = expand_conv(x,k,2,2)
    
    
    #Second Convolutional Block
    #Depth depends on n
    for i in range(n-1):
        x = conv_block2(x,k)
        
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #Third Expansion Block
    x = expand_conv(x,k,4,2)
    
    #Third Convolutional Block 
    #Depth depends on n
    for i in range(n-1):
        x = conv_block3(x,k)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #Avg pooling to get 1*1 feature vectors
    x = AveragePooling2D(pool_size=(8,8))(x)
    x = Flatten()(x)
    
    #Classification Layer
    x = Dense(10,activation="softmax")(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model

In [76]:
#Used to create Model by calling above method

model = create_wide_residual_network(32,32,10,28)

In [77]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_230 (Conv2D)              (None, 32, 32, 16)    448         image_input[0][0]                
____________________________________________________________________________________________________
batch_normalization_197 (BatchNo (None, 32, 32, 16)    64          conv2d_230[0][0]                 
____________________________________________________________________________________________________
activation_197 (Activation)      (None, 32, 32, 16)    0           batch_normalization_197[0][0]    
___________________________________________________________________________________________

In [78]:
from keras.utils import plot_model 

In [79]:
plot_model(model,"WRN-28-10.png",show_shapes=True)

In [84]:
model.compile(loss='categorical_crossentropy',
        optimizer="Adam",
        metrics=['accuracy'])

In [91]:
#Defining Callback functions which will be called by model during runtime when specified condition satisfies
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('WRN-28-10-Without_dropout.csv')
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
model_chekpoint = ModelCheckpoint("WRN-28-10-Without_dropout.hdf5",monitor = 'val_loss',verbose = 1,save_best_only=True)

In [ ]:
#model Parameters
batch_size = 64
data_augmentation = True
epochs = 100

In [90]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,verbose=1,validation_data=(x_test,y_test),callbacks = [lr_reducer,early_stopper,csv_logger,model_chekpoint])
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=batch_size*4,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    
    
    

-------------Using Data augmentation------------
Epoch 1/100


KeyboardInterrupt: 